Dependencies of this solution:
1. Chrome browser and chromedriver all installed
2. Browser can be opened by selenium (otherwise website will block app)
3. Gmail account turns on 2 step verification and genereated App Password

In [64]:
from selenium import webdriver
import smtplib
import ssl
import getpass
import keyring
from datetime import datetime
import schedule
import time

In [39]:
url_dict = {'Nano Speedy': 'https://us.louisvuitton.com/eng-us/products/nano-speedy-monogram-010575',
            'Neverfull PM': 'https://us.louisvuitton.com/eng-us/products/neverfull-pm-monogram-007654'}

In [59]:
def check_stock(url_dict):
    instock_dict = {}
    for product_name, product_url in url_dict.items():
        with webdriver.Chrome() as driver:
            driver.get(product_url)
            in_stock_info = driver.find_element_by_class_name('sellable').text
            print(f"{product_name.capitalize()} stock status at {datetime.now().strftime('%d-%b-%Y (%H:%M:%S)')}:")
            print(in_stock_info)
            instock_dict[product_name] = in_stock_info
    return instock_dict


def obtain_email_credentials():
    email_address = input(">>> Enter your email address here: ")
    email_pass = getpass.getpass(">>> Enter your email password here: ")
    keyring.set_password("email", email_address, email_pass)
    print(">>> Password is Saved")
    return email_address


def email_status(stock_status, product, product_url, sender_address, recipient_address):
    """
    Sending self an email of product stock status
    """
    GMAIL_USERNAME = sender_address
    GMAIL_PASSWORD = keyring.get_password("email", sender_address)
    body_of_email = f"[URL]: {product_url}. \r\n\r\n [Stock Status]: {stock_status}"
    email_subject = f'{product.capitalize()} Product Available Now!!!'
    headers = "\r\n".join([f"from: {GMAIL_USERNAME}", 
                           f"subject: {email_subject}", 
                           f"to: {recipient_address}", 
                           "mime-version: 1.0", 
                           "content-type: text/html"])
    content = f"{headers}\r\n\r\n{body_of_email}" 
    print(">>> Email Composed...")
    # creates SMTP session
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(
        'smtp.gmail.com', port=465, context=context) as server:
        server.login(GMAIL_USERNAME, GMAIL_PASSWORD)
        server.sendmail(sender_address, recipient_address, content)
    print(">>> Email Sent...")


def email_product_status(instock_dict, url_dict, sender_address, recipient_address):
    for product_name, product_status in instock_dict.items():
        if "cart" in product_status.lower():
            print(f">>> Product {product_name.capitalize()} is eligible to place in cart, sending email notice")
            email_status(product_status, product_name, url_dict[product_name],
                         sender_address, recipient_address)
        else:
            print(f">>> Product {product_name.capitalize()} is not eligible to purchase.")

In [38]:
# test email login function
email_address = obtain_email_credentials()

>>> Enter your email address here: elvin.ouyang@gmail.com
>>> Enter your email password here: ········
>>> Password is Saved


In [53]:
# test stock checking function
check_stock(url_dict)

Nano Speedy stock status at 27-Sep-2020 (11:45:00):
Item Unavailable, Check Back Soon.
Neverfull PM stock status at 27-Sep-2020 (11:45:05):
Notify Me


{'Nano Speedy': 'Item Unavailable, Check Back Soon.',
 'Neverfull PM': 'Notify Me'}

In [58]:
# test email
email_status('Test', 'Test Product', 'www.testurl.com', 'elvin.ouyang@gmail.com', 'elvin.ouyang@gmail.com')

>>> Email Composed...
>>> Email Sent...


In [60]:
instock_dict = {'Nano Speedy': 'Item Unavailable, Check Back Soon.',
                'Neverfull PM': 'Notify Me'}
email_product_status(instock_dict, url_dict, 'elvin.ouyang@gmail.com', 'elvin.ouyang@gmail.com')

>>> Product Nano Speedy is not eligible to purchase.
>>> Product Neverfull PM is not eligible to purchase.
